In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from IPython.core.display import clear_output
from urllib.parse import quote

In [2]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome()

In [3]:
def expand_shadow_element(element):
  shadow_root = driver.execute_script('return arguments[0].shadowRoot', element)
  return shadow_root

In [4]:
def get_rt_movie_url(movie, year):
    clear_output(wait = True)
    
    url = r"https://www.rottentomatoes.com/search?search="+ quote(movie)
    
    driver.get(url)
    try:
        root1 = driver.find_element_by_tag_name('search-result-container')
        shadow_root1 = expand_shadow_element(root1)

        root2 = shadow_root1.find_element_by_css_selector("search-result[type='movie']")
        shadow_root2 = expand_shadow_element(root2)

        html_of_interest=driver.execute_script('return arguments[0].innerHTML',shadow_root2)
        sel_soup=BeautifulSoup(html_of_interest, 'html.parser')
        
        movie_url = sel_soup.find('media-row',{'releaseyear':year})
    except NoSuchElementException:
        return None
    
    #driver.quit()
    
    #print(type(movie_url))
    if movie_url is not None:
        print("get_rt_movie_url\nMovie:{},url:{}".format(movie,movie_url['url']))
        sleep(randint(1,3))
        return movie_url['url']
    else:
        print("Movie Url not found!")
        sleep(randint(1,3))
        return None
    
    
    

In [5]:
def get_rt_scores(movie, year):
    movie_url = get_rt_movie_url(movie, year)
    a = np.nan
    c = np.nan
    
    if movie_url is not None:
        response = get(movie_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        
        x = html_soup.find('score-board')
        
        sleep(randint(1,3))

        a = x['audiencescore']
        c = x['tomatometerscore']
        
        print("get_rt_scores\nMovie:{},critic:{},audience:{}".format(movie, c, a))
        
    return [c,a]
    

In [6]:
imdb_df = pd.read_csv('imdb_meta.csv')

In [7]:
imdb_df

,movie,year,runtime,genre,imdb,meta,imdb_votes
0,The Shawshank Redemption,1994,142 min,Drama,9.3,80,2360495
1,The Dark Knight,2008,152 min,"Action, Crime, Drama",9.0,84,2319761
2,Inception,2010,148 min,"Action, Adventure, Sci-Fi",8.8,74,2083710
3,Fight Club,1999,139 min,Drama,8.8,66,1868351
4,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,94,1839615
...,...,...,...,...,...,...,...
8887,Falling Overnight,2011,86 min,"Drama, Romance",6.7,40,1002
8888,Fremde Haut,2005,97 min,Drama,7.0,60,1002
8889,Bam gua nat,2008,144 min,Drama,7.1,73,1001
8890,Awful Nice,2013,92 min,Comedy,4.9,42,1000


In [12]:
movies = imdb_df['movie']

In [13]:
years = imdb_df['year']

In [14]:
movies

0       The Shawshank Redemption
1                The Dark Knight
2                      Inception
3                     Fight Club
4                   Pulp Fiction
                  ...           
8887           Falling Overnight
8888                 Fremde Haut
8889                 Bam gua nat
8890                  Awful Nice
8891               Crazy as Hell
Name: movie, Length: 8892, dtype: object

In [15]:
years

0       1994
1       2008
2       2010
3       1999
4       1994
        ... 
8887    2011
8888    2005
8889    2008
8890    2013
8891    2002
Name: year, Length: 8892, dtype: int64

In [22]:
t=1527

In [23]:
m,y = movies[t], str(years[t])

In [24]:
movies[t]

'Bridget Jones: The Edge of Reason'

In [25]:
print("{},{}".format(m,y))

Bridget Jones: The Edge of Reason,2004


In [26]:
url = get_rt_movie_url(m,y)

get_rt_movie_url
Movie:Bridget Jones: The Edge of Reason,url:https://rottentomatoes.com/m/bridget_jones_the_edge_of_reason


In [27]:
get_rt_scores(m,y)
#print("main\nMovie:{},critic:{},audience:{}".format(m,critic,aud))

get_rt_movie_url
Movie:Bridget Jones: The Edge of Reason,url:https://rottentomatoes.com/m/bridget_jones_the_edge_of_reason
get_rt_scores
Movie:Bridget Jones: The Edge of Reason,critic:27,audience:60


['27', '60']

In [8]:
rt_df = imdb_df.copy()

In [9]:
test_df = rt_df.sample(4)

In [ ]:
type(rt_df['year'])

In [10]:
rt_df[['rt_critic','rt_audience']] = rt_df.apply(lambda row : pd.Series(get_rt_scores(row['movie'], str(row['year']))), axis=1) 

get_rt_movie_url
Movie:Bridget Jones: The Edge of Reason,url:https://rottentomatoes.com/m/bridget_jones_the_edge_of_reason


TypeError: 'NoneType' object is not subscriptable

In [11]:
x

NameError: name 'x' is not defined

In [ ]:
test_df[['rt_critic','rt_audience']] = test_df.apply(lambda row : pd.Series(get_rt_scores(row['movie'], str(row['year']))), axis=1) 

In [ ]:
rt_df

In [ ]:
test_series

In [ ]:
test_df